In [ ]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import pyeeg
import pickle
from tqdm import tqdm
import mne
import picard
import time
import sys
import logging
import statistics
import pywt
logging.getLogger("mne").setLevel(logging.WARNING)
#тут нужно указать путь в __init__.py из папки entropy

MODULE_PATH = "C:/Users/Никита/Desktop/entropy/__init__.py"

MODULE_NAME = "entropy"

import importlib
import sys

spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module 
spec.loader.exec_module(module)

from entropy import *

import warnings
warnings.filterwarnings('ignore')


In [ ]:
#функции для извлечения фичей из сырого ээг - сигнала

def first_der(x):
    return np.mean(np.abs(x[1:]-x[0:-1]))

def second_der(x):
    return np.mean(np.abs(x[2:] - x[0:-2]))

def first_der_norm(x):
    return first_der(x/max(np.abs(x)))

def second_der_norm(x):
    return second_der(x/max(np.abs(x)))
def samp_entropy(X):
    return sample_entropy(X)

def spec_entropy(X):
    return spectral_entropy(X, 100)

def sing_entropy(X):
    return svd_entropy(X)

def petrosyan(X):
    return petrosian_fd(X)

def Hig(X):  
    return higuchi_fd(X)

def Katz(X):
    return katz_fd(X)

def power_4_8(X):
    return pyeeg.bin_power(X, [4, 8], 1000)[0][0]

def power_8_16(X):
    return pyeeg.bin_power(X, [8, 16], 1000)[0][0]

def power_16_32(X):
    return pyeeg.bin_power(X, [16, 32], 1000)[0][0]

def power_32_64(X):
    return pyeeg.bin_power(X, [32, 64], 1000)[0][0]

def freq_delta(X, Fs=100):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    delta = frequency[L*1//Fs-1: L*4//Fs]
    return np.std(delta)

def freq_theta(X, Fs=100):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    theta = frequency[L*4//Fs-1: L*8//Fs]
    return np.std(theta)

def freq_alpha(X, Fs=100):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    alpha = frequency[L*5//Fs-1: L*13//Fs]
    return np.std(alpha)

def freq_beta(X, Fs=100):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    beta = frequency[L*13//Fs-1: L*30//Fs]
    return np.std(beta)

def freq_gamma(X, Fs=100):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    gamma = frequency[L*30//Fs-1: L*50//Fs]
    return np.std(gamma)

def wwlt(X):
    coeffs = pywt.wavedec(X, 'db1', level=5)
    cA2, cD2,cD3,cD4,cD5, cD6 = coeffs
    gamma = cD5
    beta = cD4
    alpha = cD3
    theta = cD2
    return sum(gamma**2), sum(beta**2), sum(alpha**2), sum(theta**2)



functions = [np.mean,
             #np.std,
             first_der,second_der,statistics.median,
             #first_der_norm, second_der_norm,perm_entropy,app_entropy, samp_entropy,
            #spec_entropy, sing_entropy, petrosyan, Hig, Katz, power_4_8, power_8_16, power_16_32, power_32_64,
            #freq_delta, freq_theta, freq_alpha,
             freq_beta, freq_gamma]
def remove_eye_activity_ICA(data):
    good_chanels = [2, 3, 4, 5, 8, 12, 14, 32, 30, 26, 22, 21, 20, 18]
    ch_names = ["AF3", "F3", "F7", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F8", "F4","AF4"]
    EOG_chanels = [33, 34, 35, 36]
    eeg = []
    
    for ch in good_chanels:
        eeg.append(np.array(data[ch]))

    for ch in EOG_chanels:
        eeg.append(np.array(data[ch]))
        
    eeg = np.array(eeg) * 10e-6              #in volts
    
    info = mne.create_info(ch_names, 512, ch_types=["eeg"] * 14 + ["eog"] * 4)
    raw = mne.io.RawArray(eeg, info)
    
    raw_tmp = raw.copy()
    raw_tmp.filter(1, None)
    
    ica = mne.preprocessing.ICA(method="picard",
                            fit_params={"extended": True},
                            random_state=1)
    ica.fit(raw_tmp)
    #TODO: WHAT ICA'S ?
    #ica.exclude = [1]
    
    raw_corrected = raw.copy()
    
    return raw_corrected.get_data()

def remove_eye_activity_regr(data):
    good_chanels = [2, 3, 4, 5, 8, 12, 14, 32, 30, 26, 22, 21, 20, 18]
    EOG_chanels = [33, 34, 35, 36]
    eeg = []
    for ch in good_chanels:
        eeg.append(np.array(data[ch]))

    for ch in EOG_chanels:
        eeg.append(np.array(data[ch]))
    eeg = np.array(eeg) * 10e-6
    eeg1 = eeg.copy()
    info = mne.create_info(18, 512, ch_types=["eeg"] * 14 + ["eog"] * 4)
    raw1 = mne.io.RawArray(eeg, info)
    raw2 = mne.io.RawArray(eeg1, info)
    bip = np.array([[-1, 1, 0, 0], [0, 0, -1, 1]])
    raw1_eog = bip @ raw1[14:, :][0]
    raw2_eog = bip @ raw2[14:, :][0]
    raw1_eeg = raw1[:14, :][0]
    raw2_eeg = raw2[:14, :][0]
    b = np.linalg.inv(raw1_eog @ raw1_eog.T) @ raw1_eog @ raw1_eeg.T
    eeg_corrected = (raw2_eeg.T - raw2_eog.T @ b).T
    raw3 = raw2.copy()
    raw3._data[:14, :] = eeg_corrected
    return np.array(raw3.to_data_frame().T[1:])

def create_df(data, lables,subject, cut=2500, train=True, n_segments=10):
    features_all = []
    m = len(data[0])
    #нужные каналы
    
    good_chanels = [2, 3, 4, 5, 8, 12, 14, 32, 30, 26, 22, 21, 20, 18]
    ch_names = ["AF3", "F3", "F7", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F8", "F4","AF4"]
    #бьем нашу ээг на сегменты и бежим по ним в цикле 
    
    for j in range(n_segments):
        start = cut + j*(m - cut)//n_segments
        finish = cut + (j + 1)*(m - cut)//n_segments
        features = dict()
        for k in range(14):
            for func in functions:
                features[f"channel_{ch_names[k]}_{func.__name__}"] = func(data[k][start:finish])
        features['arousal'] = lables[0]
        features['valence'] = lables[1]
        features['dominance'] = lables[2]
        features['liking'] = lables[3]
        features['subject'] = subject
        features_all.append(features)
    return pd.json_normalize(features_all)


In [ ]:
datasets = []

# бежим по дериктории, применяем извлечение фичей ко всем файлам и склеиваем все фичи в один датасет

for fn in tqdm(os.listdir('data_preprocessed_python/')):
    with open('data_preprocessed_python/'+fn, 'rb') as f:
        raw = pickle.loads(f.read(), encoding='latin1')
        video = 0
        while video < 40:
            label = raw['labels'][video]
            #print('Video:', video + 1,'file:', fn)
            datasets.append(create_df(raw['data'][video], label, fn.split('.')[0]))
            video += 1

data_all = pd.concat(datasets)

In [ ]:
#wawelet_features = list(filter(lambda x: x.endswith('wwlt'), data_all.columns))

In [ ]:

#for wfeature in wawelet_features:
#    for i in range(4):
#        data_all[wfeature + f"_{i}"] = data_all[wfeature].apply(lambda x: x[i])


In [ ]:
#data_all.drop(wawelet_features, axis=1, inplace=True)

In [ ]:
#перемешиваем фичи, для улучшения обучения

data_all = data_all.sample(frac=1)
data_all
#data_all.to_excel('features_and_lables.xlsx', index=False)

In [ ]:
# получаем X и Y для обучения

X = data_all.drop(['arousal', 'valence', 'dominance', 'liking', 'subject'], axis=1)
y = data_all['arousal'].apply(lambda x: 0 if x <= 5.5 else 1)
                              #if x > 4.5 and x < 5.5 else 2)

In [ ]:
#сама модель

from catboost import Pool, cv


cv_dataset = Pool(data=X,
                  label=y)

params = {"iterations": 10000,
          "depth": 6,
          "loss_function": "Logloss",
          "custom_metric": "Accuracy",
          "verbose": False}

scores = cv(cv_dataset,
            params,
            fold_count=3,
            plot="True")

In [ ]:
from catboost import CatBoostClassifier

cat_arousal_model = CatBoostClassifier(iterations=10000, depth=6)
cat_dominance_model = CatBoostClassifier(iterations=10000, depth=6)
cat_valence_model = CatBoostClassifier(iterations=10000, depth=6)

In [ ]:
def train_model(measure, data, model):
    X = data_all.drop(['arousal', 'valence', 'dominance', 'liking', 'subject'], axis=1)
    y = data_all[measure].apply(lambda x: 0 if x <= 4.5 else 1 if x > 4.5 and x < 5.5 else 2 )
    model.fit(X, y)
    return model

In [ ]:
arousal_model = train_model('arousal', data_all, cat_arousal_model)


In [ ]:
headers = data_all.columns.values.tolist()
for i in range(len(arousal_model.feature_importances_)):
    print(headers[i],' :', arousal_model.feature_importances_[i])

In [ ]:
arousal_model.score(X, y)

In [ ]:
import pickle as pkl

pkl.dump(arousal_model, open("arousal_removed_eyes.pkl", "wb"))


In [ ]:
dominance_model = train_model('dominance', data_all, cat_dominance_model)
pkl.dump(dominance_model, open("dominance_removed_eyes.pkl", "wb"))

In [ ]:
valence_model = train_model('valence', data_all, cat_valence_model)
pkl.dump(valence_model, open("valence_removed_eyes.pkl", "wb"))

In [ ]:
with open('arousal.pkl', 'rb') as f:
    model_arousal = pkl.load(f)
with open('dominance.pkl', 'rb') as f:
    model_dominance = pkl.load(f)
with open('valence.pkl', 'rb') as f:
    model_valence = pkl.load(f)

In [ ]:
f = open('data_preprocessed_python/s01.dat', 'rb')
data = pickle.loads(f.read(), encoding='latin1')